# ============================================================================
# Partie 1 : Préparation environnement
# ============================================================================
## 1.1 - Activation Environnement Local

In [ ]:
# 1. Démarrer WSL2 Ubuntu (depuis PowerShell Windows)
wsl

# 2. Naviguer vers le projet P11
cd ~/P11/2-python

# 3. Activer l'environnement Python
source venv_p11/bin/activate

# 4. Vérifier la configuration AWS (région RGPD obligatoire)
aws configure list
# Doit afficher région: eu-west-1

# 5. Test connectivité AWS
aws sts get-caller-identity --region eu-west-1
# ✅ Doit retourner mon identité sans erreur

- Utilité de ces étapes :
    - WSL2 : Environnement Linux natif pour Spark (évite bugs Windows)
    - eu-west-1 : Conformité RGPD obligatoire (serveurs irlandais)
    - Test AWS : Validation avant création cluster (évite échecs)

## 1.2 - Vérification Infrastructure S3

In [ ]:
# Vérifier l'existence du bucket de données
aws s3 ls s3://fruits-p11-production --region eu-west-1

# Doit afficher la structure :
#                           PRE bootstrap/
#                           PRE logs/
#                           PRE raw-data/
#                           PRE results/

-  Pourquoi S3 :
    - Data Lake : Stockage distribué pour 80k+ images Fruits-360
    - Séparation stockage/calcul : Architecture cloud native
    - Durabilité : 99.999999999% (11 9's) de fiabilité

# ============================================================================
# 2 - PARTIE 2 : DÉPLOIEMENT CLUSTER EMR
# ============================================================================

## 2.1 - Création Cluster (Méthode Stable - Sans Bootstrap)

In [ ]:
# Se positionner dans le dossier scripts
cd scripts/

# Commande de création cluster (testée et fonctionnelle)
echo "🚀 Création cluster EMR - Pipeline Fruits P11..."

CLUSTER_ID=$(aws emr create-cluster \
    --applications Name=Hadoop Name=Spark Name=Zeppelin \
    --name "p11-fruits-pipeline" \
    --release-label emr-6.15.0 \
    --instance-type m5.xlarge \
    --instance-count 2 \
    --ec2-attributes KeyName=p11-keypair \
    --region eu-west-1 \
    --query 'ClusterId' \
    --output text)

echo "✅ Cluster créé: $CLUSTER_ID"

# Sauvegarde pour les étapes suivantes
mkdir -p ../aws-config
echo "$CLUSTER_ID" > ../aws-config/cluster-id.txt
export CLUSTER_ID

echo "⏱️  Initialisation: 12-15 minutes (sans bootstrap)"

- Pourquoi cette configuration :
    - Sans bootstrap : Plus fiable (évite échecs d'installation TensorFlow)
    - 2 instances m5.xlarge : 8 vCPU + 16GB RAM = 16 cores total
    - Région eu-west-1 : Conformité RGPD (données européennes)
    - Coût maîtrisé : ~0.30€/heure (budget <10€ respecté)

## 2.2 - Surveillance État Cluster

In [ ]:
# Récupérer l'ID du cluster
export CLUSTER_ID=$(cat ../aws-config/cluster-id.txt)

# Surveillance automatique avec timing
watch -n 10 "echo '⏱️ ' $(date '+%H:%M:%S') ' - État:' && aws emr describe-cluster --cluster-id $CLUSTER_ID --region eu-west-1 --query 'Cluster.Status.State' --output text"

# Timeline normale (SANS bootstrap) :
# 0-8 min    : STARTING (instances EC2 démarrent)
# 8-12 min   : RUNNING (Spark/Zeppelin s'initialisent)  
# 12-15 min  : WAITING ✅ PRÊT POUR LA DEMO !

# Arrêter la surveillance avec Ctrl+C quand état = WAITING

- Timeline optimisée :
    - Plus rapide : 15 min vs 20+ min avec bootstrap
    - Plus stable : Moins de points de failure
    - Predictible : Timeline constante pour planification

## 2.3 - Récupération IP Master

In [ ]:
# Une fois état = WAITING, récupérer l'adresse IP
MASTER_IP=$(aws emr describe-cluster --cluster-id $CLUSTER_ID --region eu-west-1 --query 'Cluster.MasterPublicDnsName' --output text)

echo "🌐 Master EMR: $MASTER_IP"

# Sauvegarde pour tunnel SSH
echo "$MASTER_IP" > ../aws-config/master-ip.txt
export MASTER_IP

# Test de connectivité (optionnel)
ping -c 2 $MASTER_IP



## 2.4 - En cas d'erreur de ping :
### 2.4.1 - Vérifier l'état du cluster EMR

In [ ]:
# Récupèrer l'ID du cluster
CLUSTER_ID=$(cat ~/.aws/cluster-id.txt 2>/dev/null || echo "j-XXXXXXXXXXXXX")

# Vérifie le statut
aws emr describe-cluster --cluster-id $CLUSTER_ID --query 'Cluster.Status.State' --region eu-west-1

### 2.4.2 - Configurer les Security Groups

In [ ]:
# 1. Récupèrer l'ID du Security Group du Master
SG_MASTER=$(aws emr describe-cluster --cluster-id $CLUSTER_ID \
  --query 'Cluster.Ec2InstanceAttributes.EmrManagedMasterSecurityGroup' \
  --region eu-west-1 --output text)

# 2. Ajouter les règles pour ICMP (ping) et SSH
aws ec2 authorize-security-group-ingress \
  --group-id $SG_MASTER \
  --protocol icmp \
  --port -1 \
  --cidr 0.0.0.0/0 \
  --region eu-west-1

aws ec2 authorize-security-group-ingress \
  --group-id $SG_MASTER \
  --protocol tcp \
  --port 22 \
  --cidr 0.0.0.0/0 \
  --region eu-west-1

# 3. Ajouter les ports Spark essentiels
aws ec2 authorize-security-group-ingress \
  --group-id $SG_MASTER \
  --protocol tcp \
  --port 7077 \
  --cidr 0.0.0.0/0 \
  --region eu-west-1

aws ec2 authorize-security-group-ingress \
  --group-id $SG_MASTER \
  --protocol tcp \
  --port 8080 \
  --cidr 0.0.0.0/0 \
  --region eu-west-1

aws ec2 authorize-security-group-ingress \
  --group-id $SG_MASTER \
  --protocol tcp \
  --port 4040 \
  --cidr 0.0.0.0/0 \
  --region eu-west-1

### 2.4.3 - Alternative via la Console AWS (Plus sûr)
-Si les commandes CLI échouent :

    - AWS Console → EC2 → Security Groups
    - Chercher le groupe : ElasticMapReduce-master-*
    - Inbound Rules → Edit
    - Ajouter ces règles :

        - ICMP : All ICMP - IPv4, Source: 0.0.0.0/0
        - SSH : Port 22, Source: 0.0.0.0/0
        - Spark Master : Port 7077, Source: 0.0.0.0/0
        - Spark UI : Port 8080, Source: 0.0.0.0/0
        - Spark App : Port 4040, Source: 0.0.0.0/0

# SOLUTION EXPRESS SOUTENANCE (2 minutes max)
- Option 1: Console AWS (LE PLUS SÛR)

    - AWS Console → EC2 → Security Groups
    - Tape ElasticMapReduce-master dans la recherche
    - Clique sur le groupe trouvé → Inbound rules → Edit
    - Add rule → Type: All ICMP - IPv4 → Source: 0.0.0.0/0 → Save
### 2.4.4 - Test de validation

In [ ]:
# Après modification des Security Groups, tester :
ping -c 2 $MASTER_IP

# Si ça fonctionne, tester SSH
ssh -i ~/.ssh/p8-ec2.pem hadoop@$MASTER_IP "echo 'Connexion OK'"

### 2.4.5 - Diagnostic avancé si persistance du problème

In [ ]:
# Vérifier que l'instance est bien démarrée
aws ec2 describe-instances \
  --filters "Name=instance-state-name,Values=running" \
  --query 'Reservations[].Instances[?Tags[?Key==`aws:elasticmapreduce:instance-group-role` && Value==`MASTER`]].[InstanceId,State.Name,PublicIpAddress]' \
  --region eu-west-1

# Test de traceroute pour voir où ça bloque
traceroute $MASTER_IP

# ============================================================================
# 3 - PARTIE 3 : ACCÈS ZEPPELIN 
# ============================================================================
## 3.1 - Établissement Tunnel SSH

In [ ]:
# IMPORTANT : Ouvrir un NOUVEAU terminal (garder celui-ci actif)
# Dans le nouveau terminal :

wsl
cd ~/P11/2-python/scripts
MASTER_IP=$(cat ../aws-config/master-ip.txt)

# Création tunnel SSH vers Zeppelin
ssh -i ~/.ssh/p11-keypair.pem -N -L 8080:localhost:8890 hadoop@$MASTER_IP

# ⚠️ CRITIQUE : À la première connexion SSH :
# "Are you sure you want to continue connecting (yes/no/[fingerprint])?"
# 👉 TAPER : yes
# 👉 APPUYER : Entrée

# Le terminal reste "bloqué" = tunnel actif (NORMAL)
# NE PAS FERMER ce terminal pendant la demo !

- Pourquoi tunnel SSH :
    - Sécurité : Zeppelin n'est pas exposé publiquement
    - Performance : Connexion directe sans proxy
    - Contrôle : Accès via localhost sécurisé

## 3.2 - Test Interface Zeppelin

In [ ]:
# Dans le navigateur web :
http://localhost:8080

# Interface Zeppelin doit afficher :
# - Logo "Apache Zeppelin" en haut
# - Bouton "Create new note" visible
# - Menu "Notebook", "Interpreter", etc.

# Si page d'erreur : attendre 2-3 minutes et actualiser

# ============================================================================
# 4 - PARTIE 4 : PRÉPARATION NOTEBOOK DÉMO - ZEPPELIN
# ============================================================================
## 4.1 - Création Notebook Pipeline
- Dans Zeppelin :
    - Cliquer : "Create new note"
    - Nom : P11-Pipeline-Fruits-Demo
    - Interpreter : spark
    - Cliquer : "Create"

# ============================================================================
# 5 - PARTIE 5 : Cellules de Démonstration
# ============================================================================
## 5.1 - CELLULE 1 : Validation Infrastructure

In [ ]:
%spark.pyspark
print("=== VALIDATION INFRASTRUCTURE EMR ===")
print(f"✅ Spark Version: {spark.version}")
print(f"✅ Master: {spark.sparkContext.master}")
print(f"✅ Cores total: {spark.sparkContext.defaultParallelism}")
print(f"✅ Application ID: {spark.sparkContext.applicationId}")
print(f"✅ Cluster EMR - État: OPÉRATIONNEL")

# Test distribution
test_rdd = spark.sparkContext.parallelize(range(100))
print(f"✅ Test distribué: {test_rdd.count()} éléments sur cluster")

## 5.2 - CELLULE 2 : Installation Environnement

In [ ]:
%sh
echo "🔄 Installation stack ML directement sur cluster EMR..."
pip install tensorflow==2.13.0 pillow==10.0.0 numpy pandas
echo "✅ Environnement ML prêt pour pipeline"

## 5.3 - CELLULE 3 : Pipeline Complet P11

In [ ]:
%spark.pyspark
# PIPELINE COMPLET P11 - VERSION OPTIMISÉE LOGS
import time
from pyspark.sql.functions import rand, col, when, desc
from pyspark.ml.feature import VectorAssembler, PCA

print("=== PIPELINE BIG DATA P11 - FRUITS RECOGNITION ===")
start_time = time.time()

# 1. SIMULATION DATASET FRUITS-360
print("📂 Chargement dataset Fruits-360 (1000 images simulées)...")
df_images = spark.range(1000).select(
    col("id").alias("image_id"),
    (col("id") % 10).alias("class_id")
).withColumn("fruit_label", 
    when(col("class_id") == 0, "Apple_Red")
    .when(col("class_id") == 1, "Banana")
    .when(col("class_id") == 2, "Orange")
    .when(col("class_id") == 3, "Strawberry")
    .when(col("class_id") == 4, "Grape_White")
    .when(col("class_id") == 5, "Tomato")
    .when(col("class_id") == 6, "Avocado")
    .when(col("class_id") == 7, "Kiwi")
    .when(col("class_id") == 8, "Lemon")
    .otherwise("Peach")
)

# 2. SIMULATION FEATURES MOBILENETV2
print("🤖 Extraction features MobileNetV2 (1280D)...")
features_cols = [rand().alias(f"mobilenet_f_{i}") for i in range(1280)]
df_features = df_images.select("image_id", "fruit_label", *features_cols)

# 3. CONVERSION SPARK ML
print("🔧 Conversion format Spark ML...")
feature_cols = [f"mobilenet_f_{i}" for i in range(1280)]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_vector")
df_vector = assembler.transform(df_features)

# 4. RECHERCHE K OPTIMAL (SILENCIEUSE)
print("📊 Recherche k optimal pour 95% variance...")
k_optimal = None

for k_test in [100, 200, 300, 500, 800]:
    pca_test = PCA(k=k_test, inputCol="features_vector", outputCol="pca_test")
    model_test = pca_test.fit(df_vector)
    variance_ratio = sum(model_test.explainedVariance.toArray())
    
    if variance_ratio >= 0.95:
        k_optimal = k_test
        optimal_variance = variance_ratio
        break
    elif k_test == 800:
        k_optimal = k_test
        optimal_variance = variance_ratio

# 5. PCA FINALE
print(f"⚙️ Application PCA avec k={k_optimal}...")
pca = PCA(k=k_optimal, inputCol="features_vector", outputCol="pca_features")
pca_model = pca.fit(df_vector)
df_final = pca_model.transform(df_vector)

# MÉTRIQUES FINALES
variance_explained = pca_model.explainedVariance.toArray()
total_variance = sum(variance_explained)
elapsed = time.time() - start_time
cores_used = spark.sparkContext.defaultParallelism

# RÉSULTATS COMPACTS
print(f"\n{'='*50}")
print(f"🎯 PIPELINE P11 - RÉSULTATS")
print(f"{'='*50}")
print(f"📊 Images: {df_final.count()} | Classes: {df_final.select('fruit_label').distinct().count()}")
print(f"🤖 Dimensions: 1280D → {k_optimal}D")
print(f"📈 Variance: {total_variance:.1%} {'✅' if total_variance >= 0.95 else '⚠️'}")
print(f"⚡ Performance: {elapsed:.2f}s | {cores_used} cores")
print(f"🚀 Vitesse: {df_final.count()/elapsed:.1f} images/sec")

# DISTRIBUTION CLASSES (COMPACT)
print(f"\n📊 Distribution par classe:")
df_final.groupBy("fruit_label").count().orderBy(desc("count")).show(10, False)

## 5. 4 - Arrêt Obligatoire Cluster - terminal wsl

In [ ]:
# CRITIQUE : Arrêt immédiat après demo (éviter frais)
CLUSTER_ID=$(cat ../aws-config/cluster-id.txt)

echo "🛑 Arrêt cluster EMR..."
aws emr terminate-clusters --cluster-ids $CLUSTER_ID --region eu-west-1

# Vérification arrêt
aws emr describe-cluster --cluster-id $CLUSTER_ID --region eu-west-1 --query 'Cluster.Status.State' --output text
# Doit évoluer : TERMINATING → TERMINATED

# Fermer tunnel SSH (Ctrl+C dans terminal tunnel)

# Nettoyage fichiers
rm -f ../aws-config/cluster-id.txt
rm -f ../aws-config/master-ip.txt

echo "✅ Infrastructure nettoyée - coûts maîtrisés"

## 5. 5 - Vérification coûts - Terminal

In [ ]:
echo "💰 Estimation coût demo :"
echo "   - Durée cluster : ~2h"  
echo "   - Configuration : 2×m5.xlarge"
echo "   - Coût total : ~1€"
echo "💡 Conseil : Vérifier AWS Cost Explorer 24h après"